In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix , mean_absolute_error , mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import sklearn.datasets as DT
import scipy.linalg as la

# Ex1

In [2]:
D = pickle.load(open('A45415_Q001_data.p', 'rb'))
print(D.keys())


x = D['X']*1.
y = D['y']
print(x.shape)
print(y.shape)

X1 = np.vstack((np.ones(2124),x))
yb = (y > 0)*2-1

dict_keys(['y', 'X'])
(2, 2124)
(2124,)


## A)

- [1,0,0,0]

In [3]:
Rx = np.dot(X1, X1.T)
ryx=np.dot(X1,yb.T)
w=np.dot(la.pinv(Rx),ryx)

print('coefs' , np.round(w, 3))

ye = np.dot(w.T,X1)
yeb=(ye>0)*2-1

MC = confusion_matrix(yb,yeb)
print('MAE', np.round(mean_absolute_error(y,ye), 3))

coefs [-0.001  0.292  0.041]
MAE 0.527


## B)
- [0,0,1,0]

In [4]:
x1 = -np.ones((2,2124))
wx = [0.00, 0.53, 0.85]

for i in range(2):
    for j in range(2124):
        x1[i,j] = wx[0]+wx[1]*x[0,j]+wx[2]*x[1,j]


Y1 = -np.ones((2,2124))

for i in range(2):
	Y1[i,y==i] = 1

#estimação a partir de um polinomio
values = np.vstack((np.zeros(x1.shape[1]),x1))
Rx = np.dot(values,values.T)
rxy = np.dot(values,Y1.T)
w = np.dot(la.pinv(Rx),rxy)


Y1e = np.dot(w.T,values)#Y1 estimado
y1e = np.argmax(Y1e,axis=0)

MC = confusion_matrix(y,y1e)

print(MC,'\n')

TN = MC[0][0]
FN = MC[1][0]
TP = MC[1][1]
FP = MC[0][1]

print('falsos alarmes', round( FP/(FP+TN), 3))
print('precision ' , round(TP/(TP+FP),3))
print('acertos' , TN + TP)


[[ 557  297]
 [ 166 1104]] 

falsos alarmes 0.348
precision  0.788
acertos 1661


## C) 
- [0,0,0,1]

In [5]:
w = np.array([0.00, 0.79, 0.61])
yb = (y > 0)*2-1
y1e = np.dot(w.T,X1)

yeb = (y1e>0)*2-1

MC = confusion_matrix(yb,yeb)

recall = MC[1][1] / (MC[1][1] + MC[1][0])
print(round(recall,3))
print(MC)


0.949
[[ 647  207]
 [  65 1205]]


# 2)

In [6]:
D = pickle.load(open('A45415_Q002_data.p', 'rb'))
print(D.keys())
x=D['x']*1.
y=D['y']
fd=D['folds']
print(x.shape, y.shape, fd.shape)
X1 = x[fd==0];X1 = X1[:,np.newaxis]
X2 = x[fd==1];X2 = X2[:,np.newaxis]
y1 = y[fd==0];y1 = y1[:,np.newaxis]
y2 = y[fd==1];y2 = y2[:,np.newaxis]

dict_keys(['x', 'y', 'folds'])
(162,) (162,) (162,)


## A)
- [0,0,0,1]

In [7]:
trainFold = X1
testFold = X2

poly=PolynomialFeatures(3,include_bias=False).fit(trainFold)
X1a=poly.transform(trainFold)
X2a=poly.transform(testFold)
rl=LinearRegression().fit(X1a,y1)
# i
print(rl.intercept_)
# ii
y2e =rl.predict(X2a)
print(rl.score(X2a, y2))


[-2.33567022]
-667.1478112280153


## B)
- [0,1,0,0]

In [8]:
trainFold = X1
testFold = X2

poly=PolynomialFeatures(4,include_bias=False).fit(X1)
X1a=poly.transform(X1)
X2a=poly.transform(X2)

rl=LinearRegression().fit(X1a,y1) #treinoooo

y2e =rl.predict(X2a) #teste
print(mean_squared_error(y2,y2e))


y1e =rl.predict(X1a) #treino
print(round(mean_squared_error(y1,y1e),2))

243.78856556750642
0.03


## C)
- [0,0,0,1]

In [9]:
Xc = x[:81]
Xc = Xc[:,np.newaxis]
yc = y[:81] 
yc = yc[:,np.newaxis]
poly=PolynomialFeatures(4,include_bias=False).fit(Xc)
lr_a=LinearRegression().fit(Xc,yc)
print(lr_a.score(Xc,yc))

poly=PolynomialFeatures(4,include_bias=False).fit(X1)
lr_b=LinearRegression().fit(X1,y1)
print(lr_b.score(X1,y1))

w = lr_b.coef_
print(w.max())

0.0008865256513370667
0.7641831088278017
-0.48604938226930794


# 3)

In [10]:
D = DT.load_diabetes()

X = D.data
Y = D.target

x_treino = X[:254]
y_treino = Y[:254]

x_teste = X[254:]
y_teste = Y[254:]

## A)
- [0,0,1,0]

In [11]:
poly = PolynomialFeatures(degree = 3, include_bias = False).fit(x_treino)
X1a = poly.transform(x_treino)
X2a = poly.transform(x_teste)


rl = Lasso(random_state=42, alpha=0.01).fit(X1a,y_treino)

print('R2 (treino:',rl.score(X1a,y_treino))
print('R2 (teste) :',rl.score(X2a,y_teste))
y1e=rl.predict(X1a)
y2e=rl.predict(X2a)
print("MEAN SQUARED ERROR", np.round(mean_squared_error(y_teste,y2e),2))

w2 = rl.coef_ #ajuda no C

R2 (treino: 0.5246288303594318
R2 (teste) : 0.5330047259317208
MEAN SQUARED ERROR 2890.32


## B)
- [0,0,0,1]

In [12]:
poly = PolynomialFeatures(degree = 2, include_bias = False).fit(x_treino)
X1a = poly.transform(x_treino)
X2a = poly.transform(x_teste)

rl = LinearRegression().fit(X1a,y_treino)

print(rl.score(X2a,y_teste))
ytestPred = rl.predict(X2a)
print(mean_squared_error(y_teste,ytestPred))

0.3947404649086055
3746.0692184207287


## C)
- [1,0,0,0]

In [13]:
w = rl.coef_
print(w.shape) #Alinea A 0001
print(w2.shape)

(65,)
(285,)


# Out

In [24]:
dictionary = pickle.load(open('A45415_Ficha4_Respostas.p', 'rb'))
dictionary.keys()

dictionary['Q001'][0,:] = np.array([1,0,0,0])
dictionary['Q001'][1,:] = np.array([0,0,1,0])
dictionary['Q001'][2,:] = np.array([0,0,0,1])

dictionary['Q002'][0,:] = np.array([0,0,0,1])
dictionary['Q002'][1,:] = np.array([0,1,0,0])
dictionary['Q002'][2,:] = np.array([0,0,0,1])

dictionary['Q003'][0,:] = np.array([0,0,1,0])
dictionary['Q003'][1,:] = np.array([0,0,0,1])
dictionary['Q003'][2,:] = np.array([1,0,0,0])

pickle.dump(dictionary, open('A45415_Ficha4_Respostas.p', 'wb'))

In [25]:
dictionary = pickle.load(open('A45415_Ficha4_Respostas.p', 'rb'))
dictionary['Q001']

array([[1., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])